In [ ]:
cd 60205048_이혜연/StoryGen

In [ ]:
from requests import request, post
import time
from glob import glob
import subprocess
import os
import shutil

os.environ['CUDA_VISIBLE_DEVICES']='0'
url = 'http://39.118.126.56:17864/storygen'
# url = 'http://61.72.182.186:5000/storygen'

cnt = 0
while cnt == 0:
    headers = {
        'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    
    resp = request('GET', url)
    print(resp.status_code)
    
    if resp.status_code == 200:
        inference_path = '/home/user/60205048_이혜연/StoryGen/inference'        
        folder_list = ['image', 'prompt', 'question', 'subtitle', 'title']
        for folder in folder_list:
            if os.path.exists(os.path.join(inference_path, folder)):
                shutil.rmtree(os.path.join(inference_path, folder))
            os.mkdir(os.path.join(inference_path, folder))

        story_prompt = resp.text.split(',')[0]
        num_scenes = resp.text.split(',')[1]
        print(story_prompt, num_scenes)
        
        subprocess.run(['python', 'inference.py', '--story_prompt', story_prompt, '--num_scenes', num_scenes])

        title_file_path = os.path.join(inference_path,'title/title.txt')
        title_file = open(title_file_path, 'r')
        title = title_file.readline()
        title_file.close()
        data = {'type':'0', 'title':title}
        resp_post = request('POST', url, data = data)
        print('title', resp_post.status_code)

        # txt-image 
        img_folder_path = os.path.join(inference_path, 'image')
        img_file_path_list = sorted([os.path.join(img_folder_path, file_name) for file_name in os.listdir(img_folder_path)])
        prompt_folder_path = os.path.join(inference_path, 'prompt')
        prompt_file_path_list = sorted([os.path.join(prompt_folder_path, file_name) for file_name in os.listdir(prompt_folder_path)])
        subtitle_folder_path = os.path.join(inference_path, 'subtitle')
        subtitle_file_path_list = sorted([os.path.join(subtitle_folder_path, file_name) for file_name in os.listdir(subtitle_folder_path)])
        for img_file_path, prompt_file_path, subtitle_file_path, i in zip(img_file_path_list, prompt_file_path_list, subtitle_file_path_list, range(len(img_file_path_list))):
            print(i, img_file_path, subtitle_file_path, prompt_file_path)
            # image
            img_file = open(img_file_path, 'rb')
            files = {'file':img_file}
            # prompt
            prompt_file = open(prompt_file_path, 'r')
            prompt = prompt_file.readline()
            prompt_file.close()
            # subtitle
            subtitle_file = open(subtitle_file_path, 'r')
            subtitle = subtitle_file.readline()
            subtitle_file.close()
            # request 
            data = {'type':'1', 'prompt':prompt, 'subtitle':subtitle, 'num':str(i)}
            resp_post = request('POST', url, files = files, data = data)
            print('text-image', resp_post.status_code)
        
        # question 
        question_folder_path = os.path.join(inference_path, 'question')
        question_file_path_list = sorted([os.path.join(question_folder_path, name) for name in os.listdir(question_folder_path)])
        for question_file_path, i in zip(question_file_path_list, range(len(question_file_path_list))):
            print(i, question_file_path)
            question_file = open(question_file_path, 'r')
            question = question_file.readline()
            question_upload = {'type':'2', 'question':question, 'num':str(i)}
            question_file.close()
            resp_post = request('POST', url, data = question_upload)
            print('question', resp_post.status_code)
        
    time.sleep(5)
        
